# Postgres Date Function

#### Extract part of the timestamp with DATE_TRUNC()
<pre>SELECT DATE_TRUNC('month','2015-04-12 14:44:18'::date)</pre> 

#### Find events relative to the present time with NOW() and CURRENT_DATE
<pre>
SELECT CURRENT_DATE, 
CURRENT_DATE - interval '12 hours' as "twelve hours ago", 
NOW(), NOW()::date - interval '1 month' as "one month ago"
</pre>

#### Isolate hour-of-day and day-of-week with EXTRACT
<pre>SELECT EXTRACT(dow FROM '2015-02-12'::date)</pre>

#### Calculate the difference between two timesetamps with  AGE, DATE_PART, EXTRACT
<pre>
SELECT '2019-01-01'::date - now()::date as diff_day, 
AGE(now()::date, '2019-01-01'::date) as age_date, 
AGE('2019-01-01'::date) as age_until_now, 
(-DATE_PART('epoch', '2019-01-01'::date) + DATE_PART('epoch', now()::date))/(60*60*24) as date_part_diff_days,
(-extract(epoch from '2019-01-01'::date) + extract(epoch from now()::date))/(60*60*24) as extract_diff_days
</pre>

#### Between is Inclusive
	1. Expected:
	SELECT * FROM Cases WHERE created_at BETWEEN the beginning of '2013-05-01' AND the end of '2013-05-01'

	2. Reality:
	SELECT * FROM Cases WHERE created_at BETWEEN '2013-05-01 00:00:00+00000' AND '2013-05-01 00:00:00+00000'

	3. Equal:
	SELECT * FROM Cases WHERE created_at = '2013-05-01 00:00:00+00000'

	4. Suggested: # if you still want to use Between 
	SELECT * FROM Cases WHERE (created_at >= '20130501' AND created_at < '20130502')

#### Example: How long does it take users to complete their profile each month on average?

    Note: extract(epoch 'timestamp' or 'interval') | http://www.postgresqltutorial.com/postgresql-extract/
<pre>
SELECT DATE_TRUNC('month',started_at) AS month,
       EXTRACT(EPOCH FROM AVG(AGE(ended_at,started_at))) AS avg_seconds
FROM modeanalytics.profile_creation_events 
GROUP BY 1
ORDER BY 1  
</pre>

# Others

#### Generate series for daily/monthly reporting needs
<pre>
WITH range_values AS (
  SELECT date_trunc('week', min(created_at)) as minval,
         date_trunc('week', max(created_at)) as maxval
  FROM users),

week_range AS (
  SELECT generate_series(minval, maxval, '1 week'::interval) as week
  FROM range_values
),

weekly_counts AS (
  SELECT date_trunc('week', created_at) as week,
         count(*) as ct
  FROM users
  GROUP BY 1
)

SELECT week_range.week,
       weekly_counts.ct
FROM week_range
LEFT OUTER JOIN weekly_counts on week_range.week = weekly_counts.week;
</pre>


# Analysis

#### Get the first row per group
<pre>
WITH _events (
  SELECT *, 
  row_number() OVER (PARTITION BY  user_id ORDER BY created_at DESC) AS row_number
  FROM events
  WHERE day = '2018-01-01'::date
)

SELECT *
FROM _events
WHERE row_number = 1
</pre>

#### Avoid Gaps in Data
<pre>
WITH days AS (
  SELECT generate_series(
    DATE_TRUNC('day', NOW()) - '30 days'::interval,
    DATE_TRUNC('day', NOW()),'1 day'::interval
  ) AS day
)

SELECT days.day, COALESCE(COUNT(users.id),0)
FROM days
LEFT JOIN users ON DATE_TRUNC('day', users.created_at) = days.day
GROUP BY 1
</pre>

#### Type Casting
<pre>
-- Cast text to boolean
select 'true'::boolean;

-- Cast float to integer
select 1.0::integer;

-- Cast integer to float
select '3.33'::float;
select 10/3.0; -- This will return a float too

-- Cast text to integer
select '1'::integer;

-- Cast text to timestamp
select '2018-01-01 09:00:00'::timestamp;

-- Cast text to date
select '2018-01-01'::date;

-- Cast text to interval
select '1 minute'::interval;
select '1 hour'::interval;
select '1 day'::interval;
select '1 week'::interval;
select '1 month'::interval;
</pre>

#### Compare Two Values when One is Null
<pre>
SELECT count(1)
FROM items
WHERE width is distinct from height;
</pre>

<pre>
SELECT count(1)
FROM items
WHERE width != height or width is Null or height is Null
</pre>

#### Case
<pre>
SELECT
  CASE
    WHEN precipitation = 0 THEN 'none'
    WHEN precipitation <= 5 THEN 'little'
    WHEN precipitation > 5 THEN 'lots'
    ELSE 'unknown'
  END AS amount_of_rain
FROM weather_data;
</pre>

#### Windows Function => Cumulative Sum Total 
<pre> # every day new generated users ... 
WITH daily_new_users AS (
  SELECT DATE_TRUNC('day', created_at) AS day, 
  COUNT(DISTINCT userid) AS daily_users
  FROM users
  GROUP BY 1
)
# rolling 7 days sum
SELECT day, daily_users, 
SUM(daily_users) OVER (ORDER BY day ROWS BETWEEN 6 UNBOUNDED PRECEDING AND CURRENT ROW) as cum_users
FROM daily_new_users
</pre>

#### Windows Function => Day over Day Compare (use lag or left join + where)
<pre> 
WITH daily_new_users AS ( # every day new generated users ... 
  SELECT DATE_TRUNC('day', created_at) AS day, 
  COUNT(DISTINCT userid) AS daily_users
  FROM users
  GROUP BY 1
)
# day to day compare + lag
SELECT day, daily_users, 
TO_CHAR((daily_users - lag(daily_users, 1) over(order by day))*100/lag(daily_users, 1) over(order by day)),'999D99%') as daily_users_change_rate
FROM daily_new_users
</pre>
<pre> 
WITH daily_new_users AS ( # every day new generated users ... 
  SELECT DATE_TRUNC('day', created_at) AS day, 
  COUNT(DISTINCT userid) AS daily_users
  FROM users
  GROUP BY 1
)
# day to day compare + left join
SELECT day, daily_users, TO_CHAR((u1.daily_users - u2.daily_users)*100.00/u2.daily_users, '999D99%') as daily_users_change_rate
FROM daily_new_users u1 left join daily_new_users 
on u2.day >= u1.day -  INTERVAL '1 Days' AND u2.day < u1.day
GROUP BY u1.day
</pre>
